# 1 Introduction
First, we have to activate the environment. The first run may take a while as external libraries will be downloaded and compiled.

In [1]:
Pkg.activate(".")

  Activating project at `~/code/OpenScience21.jl`


In [2]:
include("src/Challenge/init.jl")

evaluate (generic function with 1 method)

In [3]:
function generate_ansact_param(step_qc_exp, maxItr=1000)
    # Generate step qc
    step_qc = generate_empty_circuit()
    ansact = generate_ansact()
    # append
    append!(step_qc, step_qc_exp)
    append!(step_qc, ansact)
    length(getparameters(ansact))
    # Add measures
    addMeasuresOS(step_qc)

    # Set random parameters
    start_params = getRandParameters(step_qc)
    setparameters!(ansact, start_params)


    loss(params) = loss_expected_zero_state(execute(backend, step_qc, params))
    dloss(params) = real(loss'(params))
    of = OptimizationFunction(false, (x) -> (loss(x), dloss(x)), loss)

    val, xparams, itr = gradientDescent(of, start_params, α=0.01, maxItr=maxItr,
                                  argsArePeriodic=true, isExpectedZero=true, ϵ=1e-4, debug=false, useBigValInc=true)

    return xparams
end

generate_ansact_param (generic function with 2 methods)

In [4]:
trotter_steps = 10
trotter_steps_arr = [2, 2, 2, 2, 2]

5-element Vector{Int64}:
 2
 2
 2
 2
 2

In [5]:

errs = []
errsai = []
ts = [π/4, π/2, 3*π/4, π]
#ts = [π]
best_param = []
best_qc = nothing
for t in ts
    println("Start for t=$t")
    qc, params, params2, params3 = generate_circuit(trotter_steps, trotter_steps, t)

    full_qc = generate_empty_circuit()
    full_qc.x([3, 5])
    append!(full_qc, qc)
    addMeasuresOS(full_qc)

    # caclulate the end state
    uerr = check_simulation_err(full_qc, t)

    ###########################################################################
    do_iter = 0
    do_check = true
    first_step = true
    inv_ansact = []
    for st in trotter_steps_arr
        step_qc_exp, _, _, _ = generate_circuit(trotter_steps, st, t, params, params2, params3, init=first_step)
        #step_qc_expmat = tomatrix(step_qc)
        do_iter += st
        println("Start AI TS $do_iter")

        if first_step
            opt_step_qc_exp = step_qc_exp
        else
            opt_step_qc_exp = generate_empty_circuit()
            append!(opt_step_qc_exp, inv_ansact)
            append!(opt_step_qc_exp, step_qc_exp)
        end

        # find best parameters
        best_params = generate_ansact_param(opt_step_qc_exp, 10000)

        # Generate ansact
        ansact = generate_ansact()
        setparameters!(ansact, best_params)
        inv_ansact = inv(ansact)
        bindparameters!(inv_ansact)

        # Check step reaults
        if do_check && do_iter < trotter_steps
            check_qc, _, _, _ = generate_circuit(trotter_steps, trotter_steps-do_iter, t, params, params2, params3)

            # Simulate full circuit
            check_step_qc_full = generate_empty_circuit()
            append!(check_step_qc_full, inv_ansact)
            append!(check_step_qc_full, check_qc)
            # Add measures
            addMeasuresOS(check_step_qc_full)

            # do check
            check_err = check_simulation_err(check_step_qc_full, t)
            check_err2 = check_circuits_err(full_qc, check_step_qc_full)
            println("Check value: $check_err, $check_err2")
            @assert check_err2 < 10e-2
        end

        # next initmat
        first_step = false
    end

    ###########################################################################
    addMeasuresOS(inv_ansact)
    check_err = check_simulation_err(inv_ansact, t)
    check_err2 = check_circuits_err(full_qc, inv_ansact)
    append!(errs, check_err)
    append!(errsai, check_err2)
end

Start for t=0.7853981633974483
After iteration 1000, value: 0.015599789622868959, best: 0.005586068650886377, dval: 0.02987228140663678, H: 871.3816711959496, α: 0.4640980415170238, diffx: 0.004894621514671016
After iteration 2000, value: 0.006035989605285624, best: 0.005586068650886377, dval: 0.0013380084287622136, H: 1209.5728596131887, α: 1.2496094842657737, diffx: 0.0002477094846771166
After iteration 3000, value: 0.004221548461320562, best: 0.003868500649803083, dval: 0.007535816657882252, H: 51.60057885480154, α: 1.4668517896988849, diffx: 0.06083665203832478
After iteration 4000, value: 0.0015783711776335152, best: 0.0001782002920890941, dval: 0.007454074609969478, H: 8422.59085478904, α: 1.3948004774963603, diffx: 0.0024250640298308923
After iteration 5000, value: 1.9384199386460297e-5, best: 1.9384199386460297e-5, dval: 0.010166592080290542, H: 447.6238143761011, α: 0.6120847308880527, diffx: 5.9221969959353e-6
ERROR: We are going to slow, diffx is equal to: 1.0710246547030334

In [6]:
println("==============================")
#println("The error is $(sum(errs)).")
println("The error Ai is $(mean(errsai)).")
println("==============================")

The error Ai is 0.00515130413847824.
